# Modeling
### [1] 원핫 없이 통으로 학습 & 종목명 원핫 적용 후 학습 비교
### [2] 머신러닝 => 전체 돌리는 모듈로 상위 3개 정도 & 딥러닝 모델 비교

In [2]:
import pandas as pd
import numpy as np
import os
import shutil
# from tqdm import tqdm
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.utils import to_categorical

In [11]:
stock_df = pd.DataFrame()

stock_df = pd.read_csv('../data/label_066570_LG전자.csv', index_col='Unnamed: 0')

# for min_df in files_m[:1]:
#     _df = pd.read_csv(f'../data/test_label/{min_df}', index_col='Unnamed: 0')
#     stock_df = pd.concat([stock_df, _df])

In [12]:
# 첫 5줄을 봅니다.
stock_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,...,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,등락주선,등락비율,예탁금,주식회전율,거래성립률,label
20221108,1530,91300,91300,91300,91300,0,28735,2623500000,412839,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,91300
20221108,1520,91300,91400,91300,91400,0,2900,264890000,412839,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,91400
20221108,1519,91400,91400,91300,91400,0,1785,163100000,411136,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,91400
20221108,1518,91300,91400,91200,91400,0,3707,338480000,410618,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,91400
20221108,1517,91200,91300,91100,91300,0,5195,473790000,410427,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,91400
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201030,905,86500,86500,86200,86200,0,5241,452320000,25485,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,86500
20201030,904,86300,86500,86300,86500,0,3301,285210000,20681,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,86500
20201030,903,86600,86600,86000,86300,0,11026,950630000,19751,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,86600
20201030,902,87000,87100,86500,86600,0,5881,510830000,12604,0,...,0,0.0,0,0,0,0.0,0,0.0,0.0,87100


In [13]:
def data_type_trans(stock_df):

    stock_df = stock_df.astype({'시간':'uint16'})
    stock_df = stock_df.astype({'시가':'uint32'})
    stock_df = stock_df.astype({'고가':'uint32'})
    stock_df = stock_df.astype({'저가':'uint32'})
    stock_df = stock_df.astype({'종가':'uint32'})
    stock_df = stock_df.astype({'전일대비':'int32'})
    stock_df = stock_df.astype({'거래량':'uint32'})
    stock_df = stock_df.astype({'거래대금':'uint64'})
    stock_df = stock_df.astype({'누적체결매도수량':'uint64'})
    stock_df = stock_df.astype({'상장주식수':'uint64'})
    stock_df = stock_df.astype({'시가총액':'uint64'})
    stock_df = stock_df.astype({'외국인주문한도수량':'uint64'})
    stock_df = stock_df.astype({'외국인주문가능수량':'uint64'})
    stock_df = stock_df.astype({'외국인현보유수량':'uint64'})
    stock_df = stock_df.astype({'외국인현보유비율':'float32'})
    stock_df = stock_df.astype({'수정주가일자':'uint32'})
    stock_df = stock_df.astype({'수정주가비율':'float32'})
    stock_df = stock_df.astype({'기관순매수량':'int64'})
    stock_df = stock_df.astype({'기관누적순매수량':'int64'})
    stock_df = stock_df.drop(['등락주선', '등락비율', '예탁금', '주식회전율', '거래성립률'], axis=1)
    stock_df = stock_df.astype({'label':'uint32'})

    stock_df.info()

    return stock_df

In [14]:
stock_df = data_type_trans(stock_df)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189287 entries, 20221108 to 20201030
Data columns (total 20 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   시간         189287 non-null  uint16 
 1   시가         189287 non-null  uint32 
 2   고가         189287 non-null  uint32 
 3   저가         189287 non-null  uint32 
 4   종가         189287 non-null  uint32 
 5   전일대비       189287 non-null  int32  
 6   거래량        189287 non-null  uint32 
 7   거래대금       189287 non-null  uint64 
 8   누적체결매도수량   189287 non-null  uint64 
 9   상장주식수      189287 non-null  uint64 
 10  시가총액       189287 non-null  uint64 
 11  외국인주문한도수량  189287 non-null  uint64 
 12  외국인주문가능수량  189287 non-null  uint64 
 13  외국인현보유수량   189287 non-null  uint64 
 14  외국인현보유비율   189287 non-null  float32
 15  수정주가일자     189287 non-null  uint32 
 16  수정주가비율     189287 non-null  float32
 17  기관순매수량     189287 non-null  int64  
 18  기관누적순매수량   189287 non-null  int64  
 19  label      189

In [15]:
trans = stock_df.loc[:,['고가','label']]
trans = trans.rename(columns={'label':'pct_label'}).T

In [16]:
get_trans = trans.pct_change().T.iloc[:,-1]
get_trans

20221108    0.000000
20221108    0.000000
20221108    0.000000
20221108    0.000000
20221108    0.001095
              ...   
20201030    0.000000
20201030    0.000000
20201030    0.000000
20201030    0.000000
20201030    0.000000
Name: pct_label, Length: 189287, dtype: float64

In [17]:
stock_df = pd.concat([stock_df, get_trans], axis=1)
stock_df = stock_df.drop(['label'], axis=1)
stock_df



,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
20221108,1530,91300,91300,91300,91300,0,28735,2623500000,412839,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221108,1520,91300,91400,91300,91400,0,2900,264890000,412839,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221108,1519,91400,91400,91300,91400,0,1785,163100000,411136,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221108,1518,91300,91400,91200,91400,0,3707,338480000,410618,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20221108,1517,91200,91300,91100,91300,0,5195,473790000,410427,0,0,0,0,0,0.0,0,0.0,0,0,0.001095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201030,905,86500,86500,86200,86200,0,5241,452320000,25485,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20201030,904,86300,86500,86300,86500,0,3301,285210000,20681,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20201030,903,86600,86600,86000,86300,0,11026,950630000,19751,0,0,0,0,0,0.0,0,0.0,0,0,0.000000
20201030,902,87000,87100,86500,86600,0,5881,510830000,12604,0,0,0,0,0,0.0,0,0.0,0,0,0.000000


In [18]:
stock_df['pct_label'] = stock_df['pct_label'].mul(100)
stock_df['pct_label'] = stock_df['pct_label'].round(0)
stock_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량,상장주식수,시가총액,외국인주문한도수량,외국인주문가능수량,외국인현보유수량,외국인현보유비율,수정주가일자,수정주가비율,기관순매수량,기관누적순매수량,pct_label
20221108,1530,91300,91300,91300,91300,0,28735,2623500000,412839,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221108,1520,91300,91400,91300,91400,0,2900,264890000,412839,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221108,1519,91400,91400,91300,91400,0,1785,163100000,411136,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221108,1518,91300,91400,91200,91400,0,3707,338480000,410618,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20221108,1517,91200,91300,91100,91300,0,5195,473790000,410427,0,0,0,0,0,0.0,0,0.0,0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20201030,905,86500,86500,86200,86200,0,5241,452320000,25485,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20201030,904,86300,86500,86300,86500,0,3301,285210000,20681,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20201030,903,86600,86600,86000,86300,0,11026,950630000,19751,0,0,0,0,0,0.0,0,0.0,0,0,0.0
20201030,902,87000,87100,86500,86600,0,5881,510830000,12604,0,0,0,0,0,0.0,0,0.0,0,0,0.0


In [19]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
stock_df = stock_df.drop(columns=['상장주식수', '시가총액', '외국인주문한도수량', '외국인주문가능수량', '외국인현보유수량', '외국인현보유비율', '수정주가일자', '수정주가비율', '기관순매수량', '기관누적순매수량']
            , axis=1
)

X_stock_df = stock_df.iloc[:,0:-1]
y_stock_df = stock_df.iloc[:,-1]


In [20]:
from sklearn.model_selection import train_test_split

X_stock_sc = min_max_scaler.fit_transform(X_stock_df)

X_stock_trans_df = pd.DataFrame(data=X_stock_sc, columns=X_stock_df.columns)
X_stock_trans_df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189287 entries, 0 to 189286
Data columns (total 9 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   시간        189287 non-null  float64
 1   시가        189287 non-null  float64
 2   고가        189287 non-null  float64
 3   저가        189287 non-null  float64
 4   종가        189287 non-null  float64
 5   전일대비      189287 non-null  float64
 6   거래량       189287 non-null  float64
 7   거래대금      189287 non-null  float64
 8   누적체결매도수량  189287 non-null  float64
dtypes: float64(9)
memory usage: 13.0 MB


In [21]:
X_train, X_test, y_train, y_test = train_test_split(X_stock_sc, y_stock_df, test_size=0.2, stratify=y_stock_df, random_state=42)

In [22]:
# X_stock_trans_df 데이터 변환 해줘야 함
X_stock_trans_df

,시간,시가,고가,저가,종가,전일대비,거래량,거래대금,누적체결매도수량
0,0.862826,0.121762,0.120242,0.124448,0.121003,0.0,0.029849,0.022795,0.042550
1,0.849108,0.121762,0.121107,0.124448,0.121867,0.0,0.003010,0.002299,0.042550
2,0.847737,0.122625,0.121107,0.124448,0.121867,0.0,0.001851,0.001415,0.042374
3,0.846365,0.121762,0.121107,0.123566,0.121867,0.0,0.003848,0.002939,0.042321
4,0.844993,0.120898,0.120242,0.122683,0.121003,0.0,0.005394,0.004115,0.042301
...,...,...,...,...,...,...,...,...,...
189282,0.005487,0.080311,0.078720,0.079435,0.076923,0.0,0.005442,0.003928,0.002627
189283,0.004115,0.078584,0.078720,0.080318,0.079516,0.0,0.003426,0.002476,0.002132
189284,0.002743,0.081174,0.079585,0.077670,0.077787,0.0,0.011451,0.008258,0.002036
189285,0.001372,0.084629,0.083910,0.082083,0.080380,0.0,0.006106,0.004436,0.001299


In [23]:
X_train

array([[6.25514403e-01, 9.15371330e-02, 9.08304498e-02, ...,
        4.43387112e-03, 3.25594288e-03, 4.58237073e-02],
       [7.64060357e-01, 4.43005181e-01, 4.42041522e-01, ...,
        8.12391569e-04, 8.71489797e-04, 3.86725202e-02],
       [1.78326475e-02, 2.67702936e-02, 2.59515571e-02, ...,
        4.83383372e-03, 3.24551525e-03, 4.47454224e-03],
       ...,
       [8.46364883e-01, 4.08462867e-01, 4.07439446e-01, ...,
        9.70298881e-04, 1.00800495e-03, 2.73363054e-02],
       [6.88614540e-01, 4.17098446e-01, 4.16089965e-01, ...,
        3.23086672e-04, 3.38985113e-04, 2.46787279e-02],
       [4.33470508e-01, 3.78238342e-01, 3.77162630e-01, ...,
        2.95037347e-04, 2.98751812e-04, 1.78643631e-02]])

In [24]:
# 레이블 카테고리 개수
y_stock_df.value_counts().count()

32

In [25]:
y_stock_df.value_counts()

0.0     122907
1.0      47692
2.0      10823
3.0       3434
4.0       1745
5.0        659
6.0        516
8.0        429
7.0        425
9.0        165
15.0        65
10.0        65
12.0        55
16.0        52
11.0        45
29.0        35
13.0        30
14.0        28
30.0        27
28.0        15
17.0        12
26.0        12
31.0        10
18.0        10
19.0         7
20.0         6
21.0         5
22.0         4
24.0         3
25.0         2
27.0         2
23.0         2
Name: pct_label, dtype: int64

## 3. 원-핫 인코딩

In [80]:
# 원-핫 인코딩 처리를 합니다.
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [33]:
X_train.shape

(151429, 9)

## 4. 소프트맥스

In [86]:
y_train

,0.0,1.0,2.0,3.0,4.0,5.0,6.0,7.0,8.0,9.0,10.0,11.0
20210113,0,1,0,0,0,0,0,0,0,0,0,0
20210923,1,0,0,0,0,0,0,0,0,0,0,0
20210224,1,0,0,0,0,0,0,0,0,0,0,0
20211001,1,0,0,0,0,0,0,0,0,0,0,0
20210607,0,1,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
20220124,1,0,0,0,0,0,0,0,0,0,0,0
20211028,0,1,0,0,0,0,0,0,0,0,0,0
20220425,0,1,0,0,0,0,0,0,0,0,0,0
20201030,1,0,0,0,0,0,0,0,0,0,0,0


In [81]:
len(np.max(y_train))

c:\Users\TJ\.conda\envs\tf2\lib\site-packages\numpy\core\fromnumeric.py:84: FutureWarning: In a future version, DataFrame.max(axis=None) will return a scalar max over the entire DataFrame. To retain the old behavior, use 'frame.max(axis=0)' or just 'frame.max()'
  return reduction(axis=axis, out=out, **passkwargs)


12

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

# 모델 설정
model = Sequential()
model.add(Dense(1000,  input_dim=X_stock_df.shape[1], activation='relu'))
model.add(LSTM(X_stock_df.shape[1], activation='tanh'))
model.add(Dense(len(np.max(y_train)), activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_test, y_test))

model.evaluate(X_test, y_test)[1]

In [84]:
X_train.shape

(152112, 9)

In [26]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [27]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [28]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM

X_train_lstm = X_train.reshape(X_train.shape[0], 9, 1)
X_test_lstm = X_test.reshape(X_test.shape[0], 9, 1)


In [29]:
y_train

array([[1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       ...,
       [1., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.]], dtype=float32)

In [30]:
print(y_train.shape)
print(X_train_lstm.shape)
print(X_train_lstm)

(151429, 32)
(151429, 9, 1)
[[[6.25514403e-01]
  [9.15371330e-02]
  [9.08304498e-02]
  ...
  [4.43387112e-03]
  [3.25594288e-03]
  [4.58237073e-02]]

 [[7.64060357e-01]
  [4.43005181e-01]
  [4.42041522e-01]
  ...
  [8.12391569e-04]
  [8.71489797e-04]
  [3.86725202e-02]]

 [[1.78326475e-02]
  [2.67702936e-02]
  [2.59515571e-02]
  ...
  [4.83383372e-03]
  [3.24551525e-03]
  [4.47454224e-03]]

 ...

 [[8.46364883e-01]
  [4.08462867e-01]
  [4.07439446e-01]
  ...
  [9.70298881e-04]
  [1.00800495e-03]
  [2.73363054e-02]]

 [[6.88614540e-01]
  [4.17098446e-01]
  [4.16089965e-01]
  ...
  [3.23086672e-04]
  [3.38985113e-04]
  [2.46787279e-02]]

 [[4.33470508e-01]
  [3.78238342e-01]
  [3.77162630e-01]
  ...
  [2.95037347e-04]
  [2.98751812e-04]
  [1.78643631e-02]]]


In [35]:
y_train.shape

(151429, 32)

In [37]:

# 모델 설정
model = Sequential()
model.add(LSTM(1000, input_shape=(9, 1), activation='tanh'))
model.add(Dense(32, activation='softmax')) # 답이랑 오차를 구하기 위해서 값의 총합을 1로 바꾸는 함수
model.summary()

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 모델 실행
history=model.fit(X_train_lstm, y_train, epochs=10, batch_size=200,verbose=1)


Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 1000)              4008000   
                                                                 
 dense_3 (Dense)             (None, 32)                32032     
                                                                 
Total params: 4,040,032
Trainable params: 4,040,032
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
758/758 [==============================] - 141s 184ms/step - loss: 0.9969 - accuracy: 0.6463
Epoch 2/10
758/758 [==============================] - 139s 184ms/step - loss: 0.9488 - accuracy: 0.6493
Epoch 3/10
554/758 [====================>.........] - ETA: 37s - loss: 0.9359 - accuracy: 0.6492

In [ ]:
model.evaluate(X_test, y_test)[1]

In [ ]:
model.